<a href="https://www.kaggle.com/code/aniruddhapa/insurance-claim?scriptVersionId=113258610" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-sep-2021/train.csv
/kaggle/input/tabular-playground-series-sep-2021/test.csv
/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.pipeline import Pipeline


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Embedding,  Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

from tensorflow.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from tensorflow import keras
from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

2022-12-08 07:17:58.434313: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
import pandas as pd
import numpy as np
df_train=pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
df_train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [4]:
df_test=pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
df_test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,...,0.16253,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357
1,957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,...,0.81528,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125
2,957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,...,0.81831,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797
3,957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,...,0.86559,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910
4,957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,...,0.25190,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796


In [5]:
#n_missing column has the count of total missing values in each row
df_train['n_missing']=df_train.isna().sum(axis=1)
df_test['n_missing']=df_test.isna().sum(axis=1)

In [6]:
df_train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f111,f112,f113,f114,f115,f116,f117,f118,claim,n_missing
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1,5
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1,2
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1,8


In [7]:
#Changing the datatype of target column to Object.
df_train['claim']=df_train['claim'].astype(str)

In [8]:
df_train['claim'].dtype

dtype('O')

In [9]:
#Storing all the column names in features,except claim and id.
features=[col for col in df_train.columns if col not in ['claim','id']]

In [10]:
#Creating a pipeline to fill missing values using SimpleImputer Median strategy and transforming the input data to 
#uniform distribution for better training of the model.
pipe=Pipeline([('imputer',SimpleImputer(strategy='mean',missing_values=np.nan)),
               ('scaler',QuantileTransformer(n_quantiles=64,output_distribution='uniform')),
               ('bin',KBinsDiscretizer(n_bins=64,encode='ordinal',strategy='uniform'))])

In [11]:
#Transforming the independant features
df_train[features]=pipe.fit_transform(df_train[features])

In [12]:
df_test[features]=pipe.fit_transform(df_test[features])

In [13]:
df_test[features].head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,n_missing
0,61.0,56.0,31.0,12.0,30.0,48.0,17.0,1.0,10.0,49.0,...,21.0,44.0,23.0,5.0,47.0,37.0,14.0,21.0,30.0,28.0
1,53.0,29.0,36.0,61.0,33.0,12.0,22.0,48.0,32.0,30.0,...,58.0,23.0,12.0,55.0,48.0,17.0,45.0,50.0,16.0,0.0
2,49.0,45.0,23.0,17.0,25.0,39.0,50.0,33.0,30.0,18.0,...,13.0,45.0,7.0,18.0,38.0,19.0,14.0,46.0,7.0,28.0
3,9.0,33.0,27.0,63.0,39.0,34.0,13.0,34.0,1.0,8.0,...,55.0,18.0,20.0,32.0,34.0,16.0,60.0,48.0,19.0,0.0
4,22.0,1.0,56.0,51.0,54.0,4.0,54.0,41.0,8.0,48.0,...,26.0,57.0,25.0,14.0,41.0,49.0,60.0,23.0,20.0,0.0


In [14]:
model = Sequential([
    Input(df_train[features].shape[1:]),
    Embedding(input_dim=64, output_dim=4),
    Flatten(),
    Dense(64,  activation='relu'),
    Dropout(0.5),
     Dense(32,  activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

auc = tf.keras.metrics.AUC(name='aucroc')
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=5e-4,
        decay_steps = 900,
        decay_rate= 0.9)
callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

optimizer = RMSprop(lr=5e-4, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=[auc]) 


2022-12-08 07:19:51.654744: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-08 07:19:51.658244: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-08 07:19:51.728519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 07:19:51.729303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-12-08 07:19:51.729395: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-12-08 07:19:51.762775: I tensorflow/stream_executor/platform/def

In [15]:
model.fit(x = np.float32(df_train[features]), y = np.float32(df_train.claim),
          batch_size = 1024, shuffle = True, epochs = 100,callbacks=[callback])

2022-12-08 07:19:54.595883: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 455969444 exceeds 10% of free system memory.
2022-12-08 07:19:55.162713: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-08 07:19:55.174502: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000140000 Hz


Epoch 1/100


2022-12-08 07:19:56.000343: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


 16/936 [..............................] - ETA: 6s - loss: 0.6939 - aucroc: 0.4945

2022-12-08 07:19:56.844277: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


936/936 [==============================] - 8s 7ms/step - loss: 0.5920 - aucroc: 0.7407
Epoch 2/100
936/936 [==============================] - 6s 7ms/step - loss: 0.5227 - aucroc: 0.8004
Epoch 3/100
936/936 [==============================] - 6s 7ms/step - loss: 0.5210 - aucroc: 0.8022
Epoch 4/100
936/936 [==============================] - 7s 7ms/step - loss: 0.5200 - aucroc: 0.8034
Epoch 5/100
936/936 [==============================] - 6s 7ms/step - loss: 0.5197 - aucroc: 0.8037
Epoch 6/100
936/936 [==============================] - 6s 7ms/step - loss: 0.5187 - aucroc: 0.8048
Epoch 7/100
936/936 [==============================] - 6s 6ms/step - loss: 0.5178 - aucroc: 0.8062
Epoch 8/100
936/936 [==============================] - 6s 7ms/step - loss: 0.5180 - aucroc: 0.8061
Epoch 9/100
936/936 [==============================] - 7s 7ms/step - loss: 0.5177 - aucroc: 0.8066
Epoch 10/100
936/936 [==============================] - 6s 7ms/step - loss: 0.5169 - aucroc: 0.8073
Epoch 11/100
936/936 

In [16]:
sub=pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sub.head()

,id,claim
0,957919,0.5
1,957920,0.5
2,957921,0.5
3,957922,0.5
4,957923,0.5


In [17]:
sub['claim']=model.predict(np.float32(df_test[features]))

In [18]:
sub=sub.set_index('id')

In [19]:
sub.to_csv('submission.csv')

In [20]:
ls

__notebook__.ipynb  submission.csv


In [21]:
sub.head()

,claim
id,
957919,0.570618
957920,0.134469
957921,0.717698
957922,0.132242
957923,0.103050
